# パソコンピアノ
- 録音したらrecorded_piano.wavが作られる
- 画面に何も出てこなかったら白い紙みたいなのおす
- Qで画面消える

In [ ]:
# ! pip install pygame

In [16]:
import pygame
import numpy
from pygame.locals import *
import wave
import time

# 初期設定
pygame.init()
pygame.mixer.init()

# 半音のズレが何倍か定義
onestep_pitch = 2 ** (1.0 / 12.0)

# 倍音を追加して音を生成する関数
def generate_wave(frequency, duration, sample_rate=44100):
    t = numpy.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = 0.5 * numpy.sin(2 * numpy.pi * frequency * t)  # 基本周波数
    wave += 0.25 * numpy.sin(2 * numpy.pi * 2 * frequency * t)  # 2次倍音
    wave += 0.125 * numpy.sin(2 * numpy.pi * 3 * frequency * t)  # 3次倍音
    wave += 0.0625 * numpy.sin(2 * numpy.pi * 4 * frequency * t)  # 4次倍音
    wave = numpy.int16(wave * 32767)
    wave = numpy.column_stack((wave, wave))  # ステレオ用に2次元配列に変換
    return wave

# 無音を生成する関数
def generate_silence(duration, sample_rate=44100):
    silence = numpy.zeros((int(sample_rate * duration), 2), dtype=numpy.int16)
    return silence

def play_pitch(frequency):
    sample_rate = 44100
    duration = 1.0  # 仮の長さ
    wave = generate_wave(frequency, duration, sample_rate)
    sound = pygame.sndarray.make_sound(wave)
    sound.play(-1)  # ループ再生
    return sound

# 半音上げ下げする関数を定義
def down_pitch(base_pitch):
    return int(round(base_pitch / onestep_pitch))

def up_pitch(base_pitch):
    return int(round(base_pitch * onestep_pitch))

# 各音程の周波数を定義
A4 = 440
Ais4 = up_pitch(A4)
H4 = up_pitch(Ais4)
C5 = up_pitch(H4)
Cis5 = up_pitch(C5)
D5 = up_pitch(Cis5)
Dis5 = up_pitch(D5)
E5 = up_pitch(Dis5)

Gis4 = down_pitch(A4)
G4 = down_pitch(Gis4)
Fis4 = down_pitch(G4)
F4 = down_pitch(Fis4)
E4 = down_pitch(F4)
Dis4 = down_pitch(E4)
D4 = down_pitch(Dis4)
Cis4 = down_pitch(D4)
C4 = down_pitch(Cis4)
H3 = down_pitch(C4)
Ais3 = down_pitch(H3)
A3 = down_pitch(Ais3)

# キーボードと音程を関連づける。キーボードの"d"がC4、つまりドの音など
pitchs = {}
pitchs["a"] = A3
pitchs["w"] = Ais3
pitchs["s"] = H3
pitchs["d"] = C4
pitchs["r"] = Cis4
pitchs["f"] = D4
pitchs["t"] = Dis4
pitchs["g"] = E4
pitchs["h"] = F4
pitchs["u"] = Fis4
pitchs["j"] = G4
pitchs["i"] = Gis4
pitchs["k"] = A4
pitchs["o"] = Ais4
pitchs["l"] = H4
pitchs[";"] = C5
pitchs["@"] = Cis5
pitchs[":"] = D5
pitchs["["] = Dis5
pitchs["]"] = E5

# 画面を作成
screen = pygame.display.set_mode((1200, 600))
pygame.display.set_caption("Piano")

# 画像をロード
image = pygame.image.load("キーボード.png")

# フォントを初期化
pygame.font.init()
font = pygame.font.Font(None, 36)

# 録音用のリストを初期化
recorded_waves = []

# キー押下時間を記録する辞書
key_start_times = {}
last_key_up_time = time.time()  # 最後のキーが離された時間を記録

# 無音期間を追加するかどうかのフラグ
should_add_silence = False

# メインループ
running = True
current_sounds = {}  # 現在再生中の音を管理する辞書
is_recording = False
while running:
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            pitch = pygame.key.name(event.key)
            if pitch == 'n':
                is_recording = True
                recorded_waves = []
                should_add_silence = False  # 最初の無音を追加しないようにする
            elif pitch == 'm':
                is_recording = False
                # 録音を保存
                if recorded_waves:
                    sample_rate = 44100
                    wave_data = numpy.concatenate(recorded_waves)
                    file_name = "recorded_piano.wav"
                    with wave.open(file_name, 'w') as wf:
                        wf.setnchannels(2)  # ステレオ
                        wf.setsampwidth(2)
                        wf.setframerate(sample_rate)
                        wf.writeframes(wave_data.tobytes())
            elif pitch in pitchs:
                if pitch in current_sounds:
                    current_sounds[pitch].stop()
                current_sounds[pitch] = play_pitch(pitchs[pitch])
                key_start_times[pitch] = time.time()  # キー押下時間を記録
                # 無音の期間を計算して追加
                if is_recording and should_add_silence:
                    silence_duration = time.time() - last_key_up_time
                    if silence_duration > 0:
                        recorded_waves.append(generate_silence(silence_duration))
                should_add_silence = True  # 以降の無音を追加するようにする
        elif event.type == KEYUP:
            pitch = pygame.key.name(event.key)
            if pitch in pitchs:
                if pitch in current_sounds:
                    current_sounds[pitch].stop()
                    del current_sounds[pitch]
                # キーが押されていた時間を計算
                if pitch in key_start_times:
                    duration = time.time() - key_start_times[pitch]
                    if is_recording:
                        recorded_waves.append(generate_wave(pitchs[pitch], duration))
                    del key_start_times[pitch]
                last_key_up_time = time.time()  # 最後のキーが離された時間を更新
            elif pitch == 'q':
                running = False

    # 画面を白でクリア
    screen.fill((255, 255, 255))
    # 画像を表示
    screen.blit(image, (0, 0))
    # 録音中ならば「Recording」を表示
    if is_recording:
        text = font.render("Recording", True, (255, 0, 0))
        screen.blit(text, (10, 10))

    pygame.display.flip()

pygame.quit()

オクターブ変えれるようにした

In [17]:
import pygame
import numpy
from pygame.locals import *
import wave
import time

# 初期設定
pygame.init()
pygame.mixer.init()

# 半音のズレが何倍か定義
onestep_pitch = 2 ** (1.0 / 12.0)

# 倍音を追加して音を生成する関数
def generate_wave(frequency, duration, sample_rate=44100):
    t = numpy.linspace(0, duration, int(sample_rate * duration), endpoint=False)
#     wave = 0.31 * numpy.sin(2 * numpy.pi * frequency * t)  # 基本周波数
#     wave += 0.14 * numpy.sin(2 * numpy.pi * 2 * frequency * t)  # 2次倍音
#     wave += 0.04 * numpy.sin(2 * numpy.pi * 3 * frequency * t)  # 3次倍音
#     wave += 0.1 * numpy.sin(2 * numpy.pi * 4 * frequency * t)  # 4次倍音
#     wave += 0.06 * numpy.sin(2 * numpy.pi * 5 * frequency * t)
#     wave += 0.06 * numpy.sin(2 * numpy.pi * 6 * frequency * t)
#     wave += 0.21 * numpy.sin(2 * numpy.pi * 7 * frequency * t)
#     wave += 0.07 * numpy.sin(2 * numpy.pi * 8 * frequency * t)
    wave = 0.31 * numpy.sin(2 * numpy.pi * frequency * t)  # 基本周波数
    wave += 0.14 * numpy.sin(2 * numpy.pi * 2 * frequency * t)  # 2次倍音
    wave += 0.04 * numpy.sin(2 * numpy.pi * 3 * frequency * t)  # 3次倍音
    wave += 0.1 * numpy.sin(2 * numpy.pi * 4 * frequency * t)  # 4次倍音
    wave += 0.06 * numpy.sin(2 * numpy.pi * 5 * frequency * t)  # 5次倍音
    wave += 0.06 * numpy.sin(2 * numpy.pi * 6 * frequency * t)  # 6次倍音
    wave += 0.21 * numpy.sin(2 * numpy.pi * 6 * frequency * t)  # 6次倍音
    wave += 0.07 * numpy.sin(2 * numpy.pi * 6 * frequency * t)  # 6次倍音
    wave = numpy.int16(wave * 32767)
    wave = numpy.column_stack((wave, wave))  # ステレオ用に2次元配列に変換
    return wave

# 無音を生成する関数
def generate_silence(duration, sample_rate=44100):
    silence = numpy.zeros((int(sample_rate * duration), 2), dtype=numpy.int16)
    return silence

def play_pitch(frequency):
    sample_rate = 44100
    duration = 1.0  # 仮の長さ
    wave = generate_wave(frequency, duration, sample_rate)
    sound = pygame.sndarray.make_sound(wave)
    sound.play(-1)  # ループ再生
    return sound

# 半音上げ下げする関数を定義
def down_pitch(base_pitch):
    return int(round(base_pitch / onestep_pitch))

def up_pitch(base_pitch):
    return int(round(base_pitch * onestep_pitch))


def set_pitches(base_frequency):
    global A3, Ais3, H3, C4, Cis4, D4, Dis4, E4, F4, Fis4, G4, Gis4, A4, Ais4, H4, C5, Cis5, D5, Dis5, E5, pitchs

    A4 = base_frequency
    Ais4 = up_pitch(A4)
    H4 = up_pitch(Ais4)
    C5 = up_pitch(H4)
    Cis5 = up_pitch(C5)
    D5 = up_pitch(Cis5)
    Dis5 = up_pitch(D5)
    E5 = up_pitch(Dis5)

    Gis4 = down_pitch(A4)
    G4 = down_pitch(Gis4)
    Fis4 = down_pitch(G4)
    F4 = down_pitch(Fis4)
    E4 = down_pitch(F4)
    Dis4 = down_pitch(E4)
    D4 = down_pitch(Dis4)
    Cis4 = down_pitch(D4)
    C4 = down_pitch(Cis4)
    H3 = down_pitch(C4)
    Ais3 = down_pitch(H3)
    A3 = down_pitch(Ais3)

    pitchs = {
        "a": A3, "w": Ais3, "s": H3, "d": C4, "r": Cis4, "f": D4, "t": Dis4,
        "g": E4, "h": F4, "u": Fis4, "j": G4, "i": Gis4, "k": A4, "o": Ais4,
        "l": H4, ";": C5, "@": Cis5, ":": D5, "[": Dis5, "]": E5
    }

set_pitches(440)

# 画面を作成
screen = pygame.display.set_mode((1200, 600))
pygame.display.set_caption("Piano")

# 画像をロード
image = pygame.image.load("キーボード.png")

# フォントを初期化
pygame.font.init()
font = pygame.font.Font(None, 36)

# 録音用のリストを初期化
recorded_waves = []

# キー押下時間を記録する辞書
key_start_times = {}
last_key_up_time = time.time()  # 最後のキーが離された時間を記録

# 無音期間を追加するかどうかのフラグ
should_add_silence = False

# メインループ
running = True
current_sounds = {}  # 現在再生中の音を管理する辞書
is_recording = False
while running:
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            pitch = pygame.key.name(event.key)
            if pitch == 'n':
                is_recording = True
                recorded_waves = []
                should_add_silence = False  # 最初の無音を追加しないようにする
            elif pitch in map(str,range(1,10)):
                if int(pitch)-4 > 1:
                    base_frequency = 440 * 2**(int(pitch)-5)
                    set_pitches(base_frequency)
                else:
                    base_frequency = 440 / 2**(5-int(pitch))
                    set_pitches(base_frequency)
            elif pitch == 'm':
                is_recording = False
                # 録音を保存
                if recorded_waves:
                    sample_rate = 44100
                    wave_data = numpy.concatenate(recorded_waves)
                    file_name = "recorded_piano.wav"
                    with wave.open(file_name, 'w') as wf:
                        wf.setnchannels(2)  # ステレオ
                        wf.setsampwidth(2)
                        wf.setframerate(sample_rate)
                        wf.writeframes(wave_data.tobytes())
            elif pitch in pitchs:
                if pitch in current_sounds:
                    current_sounds[pitch].stop()
                current_sounds[pitch] = play_pitch(pitchs[pitch])
                key_start_times[pitch] = time.time()  # キー押下時間を記録
                # 無音の期間を計算して追加
                if is_recording and should_add_silence:
                    silence_duration = time.time() - last_key_up_time
                    if silence_duration > 0:
                        recorded_waves.append(generate_silence(silence_duration))
                should_add_silence = True  # 以降の無音を追加するようにする
        elif event.type == KEYUP:
            pitch = pygame.key.name(event.key)
            if pitch in pitchs:
                if pitch in current_sounds:
                    current_sounds[pitch].stop()
                    del current_sounds[pitch]
                # キーが押されていた時間を計算
                if pitch in key_start_times:
                    duration = time.time() - key_start_times[pitch]
                    if is_recording:
                        recorded_waves.append(generate_wave(pitchs[pitch], duration))
                    del key_start_times[pitch]
                last_key_up_time = time.time()  # 最後のキーが離された時間を更新
            elif pitch == 'q':
                running = False

    # 画面を白でクリア
    screen.fill((255, 255, 255))
    # 画像を表示
    screen.blit(image, (0, 0))
    # 録音中ならば「Recording」を表示
    if is_recording:
        text = font.render("Recording", True, (255, 0, 0))
        screen.blit(text, (10, 10))

    pygame.display.flip()

pygame.quit()